## Calculating scalers for the 1 ts emulator

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from parflow import Run
from parflow.tools.io import read_pfb, read_clm, write_pfb
from parflow.tools.fs import mkdir
from parflow.tools.settings import set_working_directory
import subsettools as st
import hf_hydrodata as hf
import pandas as pd
import yaml

In [2]:
# Register the hydrodata pin
email = 'lecondon@email.arizona.edu' 
pin = '1234'
print('Registering ' + email + ' (PIN=' + pin + ') for HydroData download' ) #use lecondon@email.arizona.edu and 1234
hf.register_api_pin(email, pin)

Registering lecondon@email.arizona.edu (PIN=1234) for HydroData download


In [3]:
# Read in the mask file
options = {
      "dataset":"conus2_domain", "variable": "mask"}
mask = hf.get_gridded_data(options)
print(mask.shape)
print(np.sum(mask))

(3256, 4442)
7852823.0


In [4]:
# Get the mean and Stdev for static variables
#parameter_list= ['slope_x', 'slope_y', 'pf_indicator']
parameter_list = ['slope_x', 'slope_y', 'permeability_x', 
                  'permeability_y' , 'permeability_z',
                  'porosity','pf_flowbarrier', 'mannings', 
                  'specific_storage', 'sres' , 'ssat', 
                 'vg_alpha', 'vg_n']
var_ds = "conus2_domain"
means=[]
stdevs=[]
names=[]

for parameter in parameter_list:
    print(parameter)
    options = {
      "dataset":var_ds, "variable": parameter
    }
    data_temp = hf.get_gridded_data(options)
    
    if len(data_temp.shape)==2:
        #print('2D file')
        means.append(np.mean(data_temp[mask==1]))
        stdevs.append(np.std(data_temp[mask==1]))
        names.append(parameter)
    else:
        #print('3D file')
        #Get the mean for each layer
        for z in range(data_temp.shape[0]):
            print('Layer:'+ str(z))
            layer_temp=data_temp[z,:,:]
            means.append(np.mean(layer_temp[mask==1]))
            stdevs.append(np.std(layer_temp[mask==1]))
            names.append(f'{parameter}_layer{z}')


#print(means)
#print(stdevs)
#print(names)

slope_x
slope_y
permeability_x
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
permeability_y
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
permeability_z
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
porosity
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
pf_flowbarrier
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
mannings
specific_storage
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
sres
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
ssat
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
vg_alpha
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9
vg_n
Layer:0
Layer:1
Layer:2
Layer:3
Layer:4
Layer:5
Layer:6
Layer:7
Layer:8
Layer:9


In [5]:
# Make a dataframe and write it out as a csv and yaml
df=pd.DataFrame({'Name':names, 'mean': means, 'std': stdevs})
df.set_index('Name')
df.to_csv('static_scalers.csv', index=False)

# Save as YAML
with open('static_scalers.yaml', 'w') as file:
    yaml.dump(df.to_dict(orient='records'), file, sort_keys=False)